In [5]:
# System and utilities
import os
import re
import time
import io
import base64
from collections import Counter
import unicodedata
# Data manipulation and analysis
import numpy as np
import pandas as pd

# Natural Language Processing (NLP)
# import nltk
# from nltk.corpus import stopwords
import unicodedata
import re
import ast
#import spacy
#from langdetect import detect

# Machine Learning & Modeling
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Image processing and display
from PIL import Image
from IPython.display import display
from IPython.core.display import HTML

# # Custom modules
# from auto_translate import AutoTranslate
from keras.preprocessing.sequence import pad_sequences

2025-06-17 19:15:50.199648: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
PROC_DATA_PATH = '/Users/jansta/learn/DS-bootcamp/apr25_bds_rakuten_2/processed_data/'
df = pd.read_csv(PROC_DATA_PATH + 'X_train_with_labels_ext.csv') ## this is a df containing changes resulting from the simons-EN file
print(df.shape)
df.head(3)

(84916, 9)


,designation,description,productid,imageid,prdtypecode,cat_name_fr,cat_name_en,combined,comb_tokens
0,Olivia: Personalisiertes Notizbuch / 150 Seite...,NaN,3804725264,1263597046,10,Livres d'occasion,Used Books,Olivia: Personalisiertes Notizbuch / 150 Seite...,"['olivia', 'personalisiertes', 'notizbuch', 's..."
1,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,NaN,436067568,1008141237,2280,Revues et Magazines,Magazines and Journals,Journal Des Arts (Le) N° 133 Du 28/09/2001 - L...,"['journal', 'arts', 'art', 'marche', 'salon', ..."
2,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,PILOT STYLE Touch Pen de marque Speedlink est ...,201115110,938777978,50,Accessoires de jeux vidéo,Video Game Accessories,Grand Stylet Ergonomique Bleu Gamepad Nintendo...,"['stylet', 'ergonomique', 'gamepad', 'nintendo..."


In [ ]:
### Loading a stopwords numpy array
all_stopwords = np.load(PROC_DATA_PATH +'all_stopwords.npy', allow_pickle=True)
print(all_stopwords[:100]) ### first 100 stopwords

['notre' 'nicht' 'qualité' 'kein' 'style' 'ancien' 'fossi' 'starai' 'ont'
 'nach' 'blanche' 'ebbi' 'son' 'eusse' 'aurai' 'nouvelle' 'fois' 'ob' 'au'
 'col' 'five' 'nostre' 'delle' 'nei' 'sarò' 'soyons' 'gt' 'six' 'durch'
 'se' 'aveva' 'fossimo' 'noir' 'war' 'werde' 'sta' 'autre' 'sua' 'ed'
 'wir' 'bis' 'stette' 'dove' 'avez' 'furono' 'on' 'avremo' 'einen' 'lei'
 'faceva' 'hatte' 'facendo' 'avessimo' 'ancienne' 'solches' 'tue' 'class'
 'this' 'oui' 'différente' 'contre' 'gegen' 'violette' 'étais' 'warst'
 'is' 'agl' 'vostre' 'serai' 'fait' 'einigem' 'mm' 'utilisation' 'en'
 'donc' 'von' 'stia' 'fusse' 'vor' 'quanti' 'egrave' 'alles' 'sono'
 'verte' 'édition' 'savoir' 'li' 'du' 'dass' 'sto' 'siano' 'derselbe'
 'meines' 'allo' 'seront' 'avemmo' 'rsquo' 'br' 'weil' 'avuti']


### TF - IDF Vectorization setup - here it is only for exploration: 
### in future cases you have to use it in the pipeline
### correctly applied to Trained on train data and only applied to test data --> otherise: DATA LEAKAGE!

In [4]:
full = pd.concat([pd.Series(row) for row in X_train['comb_tokens']])
print(len(full))
vocabulary = set(full)
print(f"Unique tokens in 'comb_tokens': {len(vocabulary)}")

84916
Unique tokens in 'comb_tokens': 73011


In [5]:
VECTOR_SIZE = 15000 ## 9999 is likely to small this paramterer has to be explored
### one idea would be to tie it to the number of unique words in the corpus

### specity the tfidf vectorizer

In [ ]:
vectorizer = TfidfVectorizer(
        max_features=VECTOR_SIZE,  # Limit to N number of features
        ngram_range=(1,2),  # Unigrams and bigrams
        min_df=5, max_df=0.8,  # Ignore very rare and very common terms
        lowercase=True,
        stop_words=list(all_stopwords),  # Keep product-specific terms
        sublinear_tf=True,  # Better for large corpora
        norm='l2')

In [ ]:
### applying it to combined text
tfidf_vectors = vectorizer.fit_transform(X_train['combined'])
#np.save(DATA_PATH + 'tfidf_vectors.npy', tfidf_vectors.toarray())

In [ ]:
tfidf_vectors.toarray()

In [ ]:
### applying it to preprocessed tokens --> from comb_tokens column --> from -EN.ipynb notebook
tfidf_tok_vectors = vectorizer.fit_transform(X_train['comb_tokens'])
#np.save(DATA_PATH + 'tfidf_tok_vectors.npy', tfidf_tok_vectors.toarray())

In [ ]:
tfidf_tok_vectors.toarray()

In [ ]:
### HERE WE CAN TRY TO USE THE WORD2VEC MODEL
### we will use the same preprocessed tokens from the comb_tokens column
X_train['comb_tokens'] = X_train['comb_tokens'].apply(ast.literal_eval)

### this is a demo of a class added to the ../src/functions/text_helpers.py file

In [ ]:
class Word2VecEmbedder:
    '''takes a list of tokenized texts and returns their embeddings using Word2Vec'''
    def __init__(self, vector_size=300, min_count=2, workers=4):
        self.vector_size = vector_size
        self.min_count = min_count
        self.workers = workers
        self.model = None
    
    def fit(self, token_lists):
        print('Training Word2Vec...')
        #tokenized = [text.lower().split() for text in token_lists]
        self.model = Word2Vec(token_lists, vector_size=self.vector_size, 
                             min_count=self.min_count, workers=self.workers)
        return self
    
    def transform(self, token_lists):
        if self.model is None:
            raise ValueError('Model not trained. Call fit() first.')
        
        def text_to_vector(token_lists):
            vectors = [self.model.wv[word] for word in token_lists if word in self.model.wv]
            return np.mean(vectors, axis=0) if vectors else np.zeros(self.vector_size)
        
        embeddings = np.array([text_to_vector(text) for text in token_lists])
        print(f'Word2Vec shape: {embeddings.shape}')
        return embeddings
    
    def fit_transform(self, token_lists):
        return self.fit(token_lists).transform(token_lists)





In [ ]:
w2v = Word2VecEmbedder(vector_size=300, min_count=1)
w2v.fit(X_train['comb_tokens'][:1000])  # Just first 1000 rows

print(f"Vocabulary size: {len(w2v.model.wv.key_to_index)}")
print(f"Sample words: {list(w2v.model.wv.key_to_index.keys())[:10]}")

# 2. Check individual embeddings
sample_tokens = X_train['comb_tokens'].iloc[0]
print(f"Sample tokens: {sample_tokens}")
embedding = w2v.transform([sample_tokens])
print(f"Embedding shape: {embedding.shape}")
print(f"Embedding values: {embedding[0][:5]}")  # First 5 values
print(f"Is all zeros? {np.all(embedding == 0)}")

# 3. Check how many words are being found
def debug_transform(tokens):
    found_words = [word for word in tokens if word in w2v.model.wv]
    print(f"Tokens: {tokens}")
    print(f"Found in model: {found_words}")
    print(f"Missing: {[word for word in tokens if word not in w2v.model.wv]}")

debug_transform(sample_tokens)

In [ ]:
w2v = Word2VecEmbedder(vector_size=300)
# Train and transform training data
X_train_w2v = w2v.fit_transform(X_train['comb_tokens'])




In [ ]:
# TOKENIZATION COMPARISON

In [ ]:
DATASET_PERC = 0.2
VOCAB_LIMIT = 40000
MAX_SEQUENCE_LENGTH = 75




In [6]:
#df = pd.read_csv(PROC_DATA_PATH + 'X_train_with_labels_ext.csv')

print(f'Using {DATASET_PERC*100}% of the dataset for training ({int(df.shape[0]*DATASET_PERC)} rows) .')
df = df.head(int(df.shape[0]*DATASET_PERC))
## Preprocess the 'comb_tokens' column

df['comb_tokens'] = df['comb_tokens'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
all_tokens_filtered = []
for token_list in df['comb_tokens']:
    all_tokens_filtered.extend(token_list)
# Count token frequencies
token_counter = Counter(all_tokens_filtered)
# Keep only top N most frequent tokens
most_common_tokens = dict(token_counter.most_common(VOCAB_LIMIT))
vocab_filtered = {word: i+1 for i, word in enumerate(most_common_tokens.keys())}
vocab_size_filtered = len(vocab_filtered) + 1

def tokens_to_sequences_filtered(token_list):
    return [vocab_filtered[token] for token in token_list if token in vocab_filtered]
    
sequences = [tokens_to_sequences_filtered(tokens) for tokens in df['comb_tokens']]
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

Using 20.0% of the dataset for training (3396 rows) .


In [10]:
import sys
sys.path.append('../misc/')
from tokenization_qc import analyze_tokenizations


In [9]:
pwd

'/Users/jansta/learn/DS-bootcamp/apr25_bds_rakuten_2/notebooks'

In [ ]:
# OR KERAS TOKENIZATION:
def remove_stopwords(text):
    words = str(text).lower().split()
    return ' '.join([word for word in words if word not in all_stopwords])

df['keras_tokens'] = df['combined'].apply(remove_stopwords)

# Tokenize
tokenizer = Tokenizer(num_words=VOCAB_LIMIT)
tokenizer.fit_on_texts(df['keras_tokens'])
sequences = tokenizer.texts_to_sequences(df['keras_tokens'])
X = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')



In [11]:
results = analyze_tokenizations(df, MAX_SEQUENCE_LENGTH=75)

TOKENIZATION QUALITY ANALYSIS FOR CNN TEXT CLASSIFICATION

1. FILTERED TOKENIZATION (Nouns/Verbs/Adjs/Advs etc. kept)
--------------------------------------------------

2. STOPWORD REMOVAL TOKENIZATION
--------------------------------------------------

COMPARATIVE ANALYSIS

📊 VOCABULARY STATISTICS
------------------------------
                                 Metric Filtered (Nouns/Verbs) Stopword Removal
                        Vocabulary Size                  20624            29093
Effective Vocab (after num_words limit)                  20624            29093
                    Avg Sequence Length                   45.3             67.4
                    Max Sequence Length                    637              896
                    Min Sequence Length                      0                1
            Sequences > MAX_SEQ_LEN (%)                  22.9%            34.9%
                    Empty Sequences (%)                   0.1%             0.0%

📏 SEQUENCE LENGTH ANALYSIS
